In [1]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS.sort(key=len)

In [2]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 10:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
            columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [3]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
    df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].add_suffix('_run' + str(i)))
    mini_df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 main_3d_3D_in10_out25_dct_n_30_17-06-2020-08:17:49.csv
1 main_3d_3D_in10_out25_dct_n_30_17-06-2020-21:10:51.csv
2 main_3d_3D_in10_out25_dct_n_30_15-06-2020-21:51:31.csv
3 main_3d_3D_in10_out25_dct_n_30_16-06-2020-20:41:28.csv
4 main_3d_3D_in10_out25_dct_n_30_16-06-2020-08:55:34.csv


In [4]:
get_summary_df(merged)

,3d1000_mean,3d1000_std,3d560_mean,3d560_std,usingfulltestset_3d560_mean,usingfulltestset_3d560_std,usingfulltestset_3d1000_mean,usingfulltestset_3d1000_std
0,69.691576,0.724971,50.282083,0.854042,60.257154,0.647607,80.60715,0.611191


In [5]:
mini_df_lst[0]

,3d560_run0,usingfulltestset_3d560_run0,3d1000_run0,usingfulltestset_3d1000_run0
0,49.694201,60.40712,69.196116,81.103405


In [6]:
mini_df_lst[1]

,3d560_run1,usingfulltestset_3d560_run1,3d1000_run1,usingfulltestset_3d1000_run1
0,50.401054,59.687569,70.97176,79.984452


In [7]:
mini_df_lst[2]

,3d560_run2,usingfulltestset_3d560_run2,3d1000_run2,usingfulltestset_3d1000_run2
0,50.229551,60.107302,69.47945,80.214606


In [8]:
mini_df_lst[3]

,3d560_run3,usingfulltestset_3d560_run3,3d1000_run3,usingfulltestset_3d1000_run3
0,51.642595,61.298876,69.338,81.401167


In [9]:
mini_df_lst[4]

,3d560_run4,usingfulltestset_3d560_run4,3d1000_run4,usingfulltestset_3d1000_run4
0,49.443011,59.784903,69.472555,80.332118
